<a href="https://colab.research.google.com/github/jurand71/dw_matrix_road_signs/blob/master/day3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd "drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_signs"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_signs


In [9]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'],train['labels']
X_test, y_test = test['features'],test['labels']

In [6]:
y_train

array([41, 41, 41, ..., 25, 25, 25], dtype=uint8)

In [7]:
len(np.unique(y_train))

43

In [8]:
to_categorical(y_train[0])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [10]:
np.unique(y_train)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42], dtype=uint8)

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
model = Sequential([
        Conv2D(filters=65,kernel_size=(3,3),activation='relu', input_shape = input_shape),
        Flatten(data_format=None),
        Dense(num_classes,activation='softmax')            
])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 65)        1820      
_________________________________________________________________
flatten (Flatten)            (None, 58500)             0         
_________________________________________________________________
dense (Dense)                (None, 43)                2515543   
Total params: 2,517,363
Trainable params: 2,517,363
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(loss='categorical_crossentropy' ,optimizer='Adam',metrics=['accuracy'])
model.fit(X_train, y_train)

1088/1088 [==============================] - 8s 7ms/step - loss: 9.0926 - accuracy: 0.7815


In [0]:
def get_cnn_v1(input_shape,num_classes):
    return Sequential([
        Conv2D(filters=65,kernel_size=(3,3),activation='relu', input_shape = input_shape),
        Flatten(data_format=None),
        Dense(num_classes,activation='softmax')            
])
    
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy' ,optimizer='Adam',metrics=['accuracy'])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size',128),
      epochs=params_fit.get('epochs',5),
      verbose=params_fit.get('verbose',1),
      validation_data=params_fit.get('validation_data',(X_train,y_train)),
      callbacks = [tensorboard_callback]
  )

  return logdir

In [13]:
model = get_cnn_v1(input_shape,num_classes)
logdir = train_model(model,X_train, y_train,)

Epoch 1/5
272/272 [==============================] - 7s 26ms/step - loss: 40.0578 - accuracy: 0.7536 - val_loss: 0.2186 - val_accuracy: 0.9501
Epoch 2/5
272/272 [==============================] - 7s 25ms/step - loss: 0.2039 - accuracy: 0.9545 - val_loss: 0.1292 - val_accuracy: 0.9680
Epoch 3/5
272/272 [==============================] - 7s 25ms/step - loss: 0.1380 - accuracy: 0.9675 - val_loss: 0.0770 - val_accuracy: 0.9826
Epoch 4/5
272/272 [==============================] - 7s 25ms/step - loss: 0.1029 - accuracy: 0.9764 - val_loss: 0.0875 - val_accuracy: 0.9785
Epoch 5/5
272/272 [==============================] - 7s 25ms/step - loss: 0.1631 - accuracy: 0.9639 - val_loss: 0.1068 - val_accuracy: 0.9766


'logs/20200401-163007'

In [0]:
%tensorboard --logdir logs

In [0]:
def get_cnn_v2(input_shape,num_classes):
    return Sequential([
        Conv2D(filters=32,kernel_size=(3,3),activation='relu', input_shape = input_shape),
        MaxPool2D(),
        Dropout(0.5),

        Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
        MaxPool2D(),
        Dropout(0.3),

        Flatten(data_format=None),

        Dense(1024,activation='relu'),
        Dropout(0.5),
        Dense(num_classes,activation='softmax')             
])

In [22]:
get_cnn_v2(input_shape,num_classes).summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 2304)             

In [25]:
model = get_cnn_v2(input_shape,num_classes)
params_fit = {
    'epochs':25
}
logdir = train_model(model,X_train, y_train, params_fit)

Epoch 1/25
272/272 [==============================] - 6s 24ms/step - loss: 9.2984 - accuracy: 0.0626 - val_loss: 3.4899 - val_accuracy: 0.0536
Epoch 2/25
272/272 [==============================] - 6s 23ms/step - loss: 3.0708 - accuracy: 0.1759 - val_loss: 2.6747 - val_accuracy: 0.2622
Epoch 3/25
272/272 [==============================] - 6s 23ms/step - loss: 2.5142 - accuracy: 0.3100 - val_loss: 1.8980 - val_accuracy: 0.4642
Epoch 4/25
272/272 [==============================] - 6s 23ms/step - loss: 1.8665 - accuracy: 0.4485 - val_loss: 1.3086 - val_accuracy: 0.6052
Epoch 5/25
272/272 [==============================] - 6s 23ms/step - loss: 1.4173 - accuracy: 0.5672 - val_loss: 0.7266 - val_accuracy: 0.8025
Epoch 6/25
272/272 [==============================] - 6s 23ms/step - loss: 1.0134 - accuracy: 0.6898 - val_loss: 0.4462 - val_accuracy: 0.8839
Epoch 7/25
272/272 [==============================] - 6s 23ms/step - loss: 0.7744 - accuracy: 0.7601 - val_loss: 0.3482 - val_accuracy: 0.9080

In [28]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 605), started 0:15:19 ago. (Use '!kill 605' to kill it.)

<IPython.core.display.Javascript object>